![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Generate Portfolio Weights to Run a LEAN Backtest

This notebook connects to PredictNow, optimizes the portfolio weights for each rebalance, and then saves the rebalancing weights for each month into the Object Store. After you run the cells in this notebook, you can run the algorithm in `main.py`, which uses the portfolio weights from PredictNow in a LEAN backtest.

### Connect to PredictNow

In [3]:
from QuantConnect.PredictNowNET import PredictNowClient
from QuantConnect.PredictNowNET.Models import *
from time import sleep
from datetime import datetime

algorithm_start_date = datetime(2020, 2, 1)
algorithm_end_date = datetime(2024, 4, 1)

qb = QuantBook()
client = PredictNowClient("jared@quantconnect.com")
client.connected

### Upload Asset Returns

The returns file needs to have sufficient data to cover the backtest period of the algorithm in `main.py` and the in-sample backtest, which occurs before `algorithm_start_date`.

In [305]:
# Calculate the daily returns of the universe constituents.
tickers = [
    "TIP", "BWX", "EEM", "VGK", "IEF", "QQQ", "EWJ", "GLD", 
    "VTI", "VNQ", "TLT", "RWX", "SPY", "DBC", "REM", "SCZ"
]
symbols = [qb.add_equity(ticker).symbol for ticker in tickers]
df = qb.history(
    symbols, datetime(2019, 1, 1), algorithm_end_date, Resolution.Daily
).close.unstack(0)
# Save the returns data into the Object Store.
df.rename(lambda x: x.split(' ')[0], axis='columns', inplace=True)
returns_file_name = "ETF_return_Test.csv"
returns_file_path = qb.object_store.get_file_path(returns_file_name)
df.pct_change().dropna().to_csv(returns_file_path)
# Upload the returns file to PredictNow.
message = client.upload_returns_file(returns_file_path)
print(message)
# List the return files you've uploaded.
return_files = client.list_returns_files()
','.join(return_files)

### Upload Constraints

The constraints must contain a subset of the assets in the returns file. The CPO system only provides portfolio weights for assets that have constraints.

In [1]:
# Define the constraints.
constraints_by_symbol = {
    Symbol.create(ticker, SecurityType.EQUITY, Market.USA).ID: contraint
    for ticker, contraint in {
        "SPY": (0, 0.5),
        "QQQ": (0, 0.5),
        "VNQ": (0, 0.5)
    }.items()
}
# Create the constraints file.
content = "component,LB,UB"
for symbol, boundaries in constraints.items():
    content += f'\n{symbol},{boundaries[0]},{boundaries[1]}'
# Save the constraints file in the Object Store.
constraints_file_name = "ETF_constrain_Test.csv"
qb.object_store.save(constraints_file_name, content)
# Upload the constraints file to PredictNow.
constraint_file_path = qb.object_store.get_file_path(constraints_file_name)
message = client.upload_constraint_file(constraint_file_path)
print(message)
# List the constraint files you've uploaded.
constraint_files = client.list_constraint_files()
','.join(constraint_files)

### (Optional) Upload Features

In [7]:
pass

### Define the Portfolio Parameters

In [27]:
portfolio_parameters = PortfolioParameters(
    name=f"Demo_Project_{datetime.now().strftime("%Y%m%d")}",
    returns_file=returns_file_name,
    constraint_file=constraints_file_name,
    #feature_file=feature_file_name,
    max_cash=1.0,
    rebalancing_period_unit="month",
    rebalancing_period=1,
    rebalance_on="first",
    training_data_size=3,
    evaluation_metric="sharpe"
)

### Run the In-Sample Backtest

The in-sample period must end before the `set_start_date` in `main.py`. Since our algorithm does monthly rebalancing at the beginning of each month, the `training_start_date` argument should align with the start of the month and the `training_end_date` should be one day before the start date in `main.py`.

In [28]:
in_sample_result = client.run_in_sample_backtest(
    portfolio_parameters,
    training_start_date=datetime(2019, 1, 1),
    training_end_date=algorithm_start_date-timedelta(1),
    sampling_proportion=0.3,
    debug="debug"
)
print(in_sample_result)

def wait_for_backtest_to_finish(id_, sleep_seconds=60):
    job = client.get_job_for_id(id_)
    while job.status != "SUCCESS":   
        job = client.get_job_for_id(id_)
        print(job.status)
        sleep(sleep_seconds)
    return job

job = wait_for_backtest_to_finish(in_sample_result.id)
print(job)

### Run the Out-Of-Sample Backtest

The out-of-sample period should match the start and end dates of the algorithm `main.py`. It is important to keep the `training_start_date` parameters have the same format for in-sample and out-of-sample tests. For this example, we are working on a portfolio that takes monthly rebalance on the first market day of the month, so we will keep `training_start_date` to the 1st of the month in the out-of-sample test.

In [30]:
out_of_sample_result = client.run_out_of_sample_backtest(
    portfolio_parameters,
    training_start_date=algorithm_start_date,
    training_end_date=algorithm_end_date,
    debug="debug"
)
print(out_of_sample_result)

job = wait_for_backtest_to_finish(out_of_sample_result.id)
print(job)

### Get the Backtest Weights

Let's get the portfolio weights from the preceding out-of-sample backtest. These are the weights you will use to run the LEAN algorithm in `main.py`. Save the portfolio weights into the Object Store so that you can load them in the algorithm.

In [ ]:
weights_by_date = client.get_backtest_weights(
    portfolio_parameters, 
    training_start_date=algorithm_start_date,
    training_end_date=algorithm_end_date,
    debug= "debug"
)
print(weights_by_date)

# Save the weights into the Object Store.
qb.ObjectStore.Save("ETF_Weights_Test1.csv", json.dumps(weights_by_date))